In [ ]:
"""Investigate performance of bootstrap for phi(theta) = max(theta, 0)."""

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from joblib import Parallel, delayed

In [ ]:
def _phi_max(theta):
    return np.maximum(theta, 0)


def _phi_kink(theta, a=0.5):
    return theta * (theta < 0) + a * theta * (theta >= 0)

In [ ]:
def _experiment(n_obs, n_boot, theta_0, phi, alpha, rng, return_boot):
    x = rng.normal(theta_0, 1, n_obs)

    mle = phi(x.mean())

    boot_mle = np.zeros(n_boot)

    for i in range(n_boot):
        boot_idx = rng.choice(n_obs, n_obs, replace=True)
        boot_x = x[boot_idx]
        boot_mle[i] = phi(boot_x.mean())

    boot_distr = np.sqrt(n_obs) * (boot_mle - mle)

    boot_cval_lo = np.percentile(boot_distr, 100 * alpha / 2)
    boot_cval_hi = np.percentile(boot_distr, 100 * (1 - alpha / 2))

    ci_lo = mle - boot_cval_hi / np.sqrt(n_obs)
    ci_hi = mle - boot_cval_lo / np.sqrt(n_obs)

    if return_boot:
        return boot_distr

    return pd.Series(
        {
            "ci_lo": ci_lo,
            "ci_hi": ci_hi,
            "theta_0": theta_0,
            "alpha": alpha,
            "n_obs": n_obs,
            "n_boot": n_boot,
        },
    )

In [ ]:
def simulation(n_sim, n_obs, n_boot, theta_0, phi, alpha, rng):
    """Simulation."""
    return pd.concat(
        [_experiment(n_obs, n_boot, theta_0, phi, alpha, rng) for _ in range(n_sim)],
        axis=1,
    ).T

In [ ]:
# Repeat simulation for different values of theta_0 and add 0
theta_0_vals = np.linspace(-0.2, 0.2, 20)
n_obs_vals = [250, 1_000]

N_SIM = 2_000
N_BOOT = 2_000

ALPHA = 0.05

In [ ]:
res = Parallel(n_jobs=10)(
    delayed(simulation)(
        n_sim=N_SIM,
        n_obs=n_obs,
        n_boot=N_BOOT,
        theta_0=theta_0,
        phi=_phi_kink,
        alpha=ALPHA,
        rng=np.random.default_rng(),
        return_boot=False,
    )
    for theta_0 in theta_0_vals
    for n_obs in n_obs_vals
)

In [ ]:
res = pd.concat(res, axis=0)

In [ ]:
res["phi_0"] = _phi_kink(res["theta_0"])
res["covers"] = (res["ci_lo"] <= res["phi_0"]) & (res["ci_hi"] >= res["phi_0"])

In [ ]:
# Plot mean covergae by theta_0 and n_obs

res_grouped = res.groupby(["theta_0", "n_obs"])["covers"].mean().reset_index()

In [ ]:
fig = go.Figure()

for n_obs, group in res_grouped.groupby("n_obs"):
    fig.add_trace(
        go.Scatter(
            x=group["theta_0"],
            y=group["covers"],
            mode="lines",
            name=f"n_obs={n_obs}",
        ),
    )

fig.update_layout(
    title=f"Coverage of {1 - ALPHA} confidence intervals by theta_0",
    xaxis_title="theta_0",
    yaxis_title="Coverage",
    yaxis={"tickformat": ".2%"},
)

fig.show()